<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [6]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [8]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [9]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

### Track Experiments

In [10]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [11]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [3]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [1]:
#dagsh setup
import dagshub
import dagshub
dagshub.init(repo_owner='sindycee', repo_name='mlops_dagshub', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\sindy\mlops_dagshub\dagsenv\Lib\site-packages\rich\live.py:260: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=1cf0ee5e-ae12-4187-bbcb-a7bc0a92dec8&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=778d3794660efca4084f2fb5ca73824c240322427bef6ed9397cade14a666749




Accessing as sindycee

Initialized MLflow to track repo "sindycee/mlops_dagshub"

Repository sindycee/mlops_dagshub initialized!

In [12]:
# Initialize MLflow
mlflow.set_experiment("Anomaly_Detection")
# mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_tracking_uri("https://dagshub.com/sindycee/mlops_dagshub.mlflow")


for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            booster = model.get_booster()
            mlflow.xgboost.log_model(booster, "model", model_format="xgb")
        else:
            mlflow.sklearn.log_model(model, "model") 

2026/02/04 19:33:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: https://dagshub.com/sindycee/mlops_dagshub.mlflow/#/experiments/0/runs/4b92394b651647db98b261f95ab99d28
🧪 View experiment at: https://dagshub.com/sindycee/mlops_dagshub.mlflow/#/experiments/0


2026/02/04 19:34:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: https://dagshub.com/sindycee/mlops_dagshub.mlflow/#/experiments/0/runs/bf12caeef1134a57a39143eba269579d
🧪 View experiment at: https://dagshub.com/sindycee/mlops_dagshub.mlflow/#/experiments/0


2026/02/04 19:34:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: https://dagshub.com/sindycee/mlops_dagshub.mlflow/#/experiments/0/runs/72eb8834a9a74e749aff92aee245bc6b
🧪 View experiment at: https://dagshub.com/sindycee/mlops_dagshub.mlflow/#/experiments/0


2026/02/04 19:34:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/sindycee/mlops_dagshub.mlflow/#/experiments/0/runs/dfebd87a46d64a87be0a89e512931dda
🧪 View experiment at: https://dagshub.com/sindycee/mlops_dagshub.mlflow/#/experiments/0


### Register the Model

In [15]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

result = mlflow.register_model(model_uri=model_uri, name=model_name)

# with mlflow.start_run(run_id=run_id):
#     mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2026/01/14 17:54:12 WARNING mlflow.tracking._model_registry.fluent: Run with id ccaab4b9ee184c9fb3b5512f8f1b35bd has no artifacts at artifact path 'model', registering model based on models:/m-336bb95bf89c4631bd22132e187e979f instead
2026/01/14 17:54:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


### Load the Model

In [19]:
import xgboost
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
dtest = xgboost.DMatrix(X_test)
y_pred = loaded_model.predict(dtest)
y_pred[:4]

array([0.06893085, 0.00114779, 0.00363952, 0.00088863], dtype=float32)

### Transition the Model to Production

In [20]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1768438919458, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1768438919458, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='ccaab4b9ee184c9fb3b5512f8f1b35bd', run_link='', source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [43]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 